<a href="https://colab.research.google.com/github/junhansong/AI_begin/blob/master/automating_reply_generation_for_comments_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai # openai 라이브러리를 설치합니다.
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-

In [ ]:
import os

In [ ]:
from langchain.chat_models import ChatOpenAI

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

chat_model = ChatOpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Act as a kind and excellent restaurant owner. Respond to the user-written comment. For negative comments, provide a detailed apology and mention specific areas for improvement. For positive comments, express your gratitude in detail."),
    ("human", "{input_text}"),
])

chain = chat_prompt_template | chat_model | StrOutputParser() # LangChain Expression Language (LCEL)

def langchain_llm(input_text):

    output = chain.invoke({"input_text": input_text})

    return output

In [ ]:
def generate_reply(input_text):
    output = langchain_llm(input_text)
    return output

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarrr_sample_submit.txt",
    filename="tarrr_sample_submit.txt",
)

('tarrr_sample_submit.txt', <http.client.HTTPMessage at 0x7973b35dd240>)

In [ ]:
import pandas as pd

# 파일을 DataFrame으로 로드
df = pd.read_csv('tarrr_sample_submit.txt', delimiter='\t')

total = len(df)

for index, row in df.iterrows():
    comment = row['comment']
    reply = generate_reply(comment)

    print(f"[{index+1}]/[{total}]")
    print("comment : ", comment)
    print("reply : ", reply)
    print("---------------")

    if index > 3:
        break

[1]/[100]
comment :  완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌
reply :  긍정 단계: 5점
부정 단계: 1점
---------------
[2]/[100]
comment :  맛있긴 한데 양이 너무 적어서 좀... ㅠ
reply :  부정: 2점

양이 적어서 아쉽다는 내용이 있지만, 맛은 좋다는 언급이 있어 2점을 줄 수 있습니다.
---------------
[3]/[100]
comment :  완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨
reply :  긍정: 5점
---------------
[4]/[100]
comment :  한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.
reply :  긍정 단계: 5점
---------------
[5]/[100]
comment :  서빙하는 분이 좀 불친절해서 기분이 좀 그랬어요.
reply :  부정 단계: 2점
---------------
